In [ ]:
# Sentiment Analysis using Naive Bayes (IMDb Dataset)# Step 1: Import Librariesimport tensorflow_datasets as tfdsimport reimport nltkfrom nltk.corpus import stopwordsfrom sklearn.feature_extraction.text import TfidfVectorizerfrom sklearn.model_selection import train_test_splitfrom sklearn.naive_bayes import MultinomialNBfrom sklearn.metrics import classification_report, accuracy_score, confusion_matriximport seaborn as snsimport matplotlib.pyplot as pltimport numpy as npimport pickle# Step 2: Load Datasetnltk.download('stopwords')data, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)train_data, test_data = data['train'], data['test']X_train, y_train = [], []for text, label in tfds.as_numpy(train_data):    X_train.append(text.decode("utf-8"))    y_train.append(label)X_test, y_test = [], []for text, label in tfds.as_numpy(test_data):    X_test.append(text.decode("utf-8"))    y_test.append(label)# Step 3: Clean Textdef clean_text(text):    text = re.sub(r'<.*?>', '', text)    text = re.sub(r'[^a-zA-Z\s]', '', text)    text = text.lower()    stop_words = set(stopwords.words('english'))    text = ' '.join([word for word in text.split() if word not in stop_words])    return textX_train_clean = [clean_text(text) for text in X_train]X_test_clean = [clean_text(text) for text in X_test]# Step 4: Vectorizationvectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))X_train_vec = vectorizer.fit_transform(X_train_clean)X_test_vec = vectorizer.transform(X_test_clean)# Step 5: Model Trainingmodel = MultinomialNB()model.fit(X_train_vec, y_train)# Step 6: Evaluationy_pred = model.predict(X_test_vec)print("Accuracy:", accuracy_score(y_test, y_pred))print(classification_report(y_test, y_pred, target_names=['Negative', 'Positive']))sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d',            xticklabels=['Neg', 'Pos'], yticklabels=['Neg', 'Pos'])plt.xlabel("Predicted")plt.ylabel("Actual")plt.title("Confusion Matrix")plt.show()# Step 7: Explainabilityfeature_names = vectorizer.get_feature_names_out()log_probs = model.feature_log_prob_for i, label in enumerate(['Negative', 'Positive']):    top_words = np.argsort(log_probs[i])[-10:]    print(f"\nTop words for {label}:")    print([feature_names[j] for j in top_words])# Step 8: Save Model & Vectorizerwith open('nb_model.pkl', 'wb') as f:    pickle.dump(model, f)with open('tfidf_vectorizer.pkl', 'wb') as f:    pickle.dump(vectorizer, f)